In [1]:
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
import numpy as np
import os, sys
import h5py
import pandas as pd
import matplotlib.pyplot as plt

import readdata as rd

import datetime
time_origin = datetime.datetime(2010,1,1,0,0,0)
sys.path.append('/Users/ajc/Desktop/ATS/ats-repo/ats/tools/utils/')
import plot_water_table2D
sys.path.append(os.getcwd())
import SiteC_2012_2014WL 

In [3]:
reload(SiteC_2012_2014WL)

data2012 = SiteC_2012_2014WL.getData_2012()

In [4]:
reload(SiteC_2012_2014WL)
ObservedData2013 = dict()
data2013 = SiteC_2012_2014WL.getData_2013()

In [5]:
reload(SiteC_2012_2014WL)
ObservedData2014 = dict()
data2014 = SiteC_2012_2014WL.getData_2014()

In [6]:
reload(SiteC_2012_2014WL)
dataPrecip = SiteC_2012_2014WL.getDataRain()

[889.75       889.76041667 889.77083333]


In [42]:
ObservedData2012 = dict()
ObservedData2013 = dict()
ObservedData2014 = dict()

for key in data2013.keys():
    ObservedData2012[key] = data2012[key]
    ObservedData2013[key] = data2013[key]
    ObservedData2014[key] = data2014[key]

In [52]:
simulation = 'SiteC_0R1'
#simulation = 'SiteC_2R2Crr'
reload(plot_water_table2D)
path = "/Users/ajc/Projects/ATS-Data/OR-CONDO/simulations/barrow_icewedges/NGEE/SiteC_New/"
path = "/Users/ajc/Projects/ATS-Data/OR-CONDO/simulations/barrow_icewedges/NGEE/SiteC_2PW/"

Time, WaterLevelCenter, ThawDepthC, Z_c = plot_water_table2D.water_table2D(path + '%s/'%simulation,x_pos= 36)
Time, WaterLevelTrough, ThawDepthT, Z_t = plot_water_table2D.water_table2D(path + '%s/'%simulation, location='trough',x_pos=75)

Location:  center 36
('Datum (surface elevation)', array([4.76063772]))
Location:  trough 75
('Datum (surface elevation)', array([4.72353333]))


In [53]:
#cellid = 44
#metdata = rd.GetSurfVarFromVis(path + '/%s/visdump_surface_data.h5'%simulation, 'surface-precipitation_rain', cellid=cellid)

In [54]:
# correcting the elevation of the center
def OffsetData():
    for key in ObservedData2012.keys():
        if 'water_level_C40' in key:
            ObservedData2012[key + '_offset'] = np.array([d - 4.91 + Z_c for d in ObservedData2012[key]])
    for key in ObservedData2012.keys():
        if 'water_level_C39' in key:
            ObservedData2012[key + '_offset'] = np.array([d - 4.81 + Z_c for d in ObservedData2012[key]])
    for key in ObservedData2013.keys():
        if 'water_level_C39' in key:
            ObservedData2013[key + '_offset'] = np.array([d - 4.81 + Z_c for d in ObservedData2013[key]])
    for key in ObservedData2014.keys():
        if 'water_level_C39' in key:
            ObservedData2014[key + '_offset'] = np.array([d - 4.81 + Z_c for d in ObservedData2014[key]])

In [55]:
DataSim =dict()
time = [3*365*0 + t*365.25 for t in Time]

DataSim['time'] = time
DataSim['WL_center'] = WaterLevelCenter
DataSim['WL_trough'] = WaterLevelTrough
DataSim['TD_center'] = [t + Z_c for t in ThawDepthC]
DataSim['TD_trough'] = [t + Z_t for t in ThawDepthT]

Data_C = []
Data_T = []
Time_C = []
Time_T = []
for i,d in enumerate(DataSim['WL_center']):
    if d != 0:
        Data_C.append(d + Z_c)
        Time_C.append(DataSim['time'][i])
for i,d in enumerate(DataSim['WL_trough']):
    if d != 0:
        Data_T.append(d + Z_t)
        Time_T.append(DataSim['time'][i])

DataSim['water_level_center'] = np.array(Data_C)
DataSim['water_level_trough'] = np.array(Data_T)
DataSim['time_center'] = np.array(Time_C)
DataSim['time_trough'] = np.array(Time_T)

# Simulated data surface rain
#infile = h5py.File('/Users/ajc/Desktop/ATS/forcing_data/Barrow-Anna-WaterLevel/Barrow-2010_2016-updated.h5')

#DataSim['Rain_forcing'] = [r*1000*86400 for r in infile['precipitation rain [m s^-1]']]
#DataSim['Time_forcing'] = [r/86400 for r in infile['time [s]']]
OffsetData()

In [56]:
#--- Remove the assigned 100 to the missing data points

LC39 = len(ObservedData2012['water_level_C39_offset'])
mask = np.ones(LC39, dtype=bool)

for i,d1 in enumerate(ObservedData2012['water_level_C39_offset']):
    if d1 > 10:
        mask[i] = False

ObservedData2012['water_level_C39_mask'] = ObservedData2012['water_level_C39_offset'][mask]
ObservedData2012['time_C39_mask'] = ObservedData2012['time_C39'][mask]

#-----------------------------------------------------------------


# Average the data at the two center wells
assert len(ObservedData2012['water_level_C39']) == len(ObservedData2012['water_level_C40_offset'])
data = []
index = np.where(ObservedData2012['water_level_C39_offset'] > 10)[0]

mid_value = 0.5*(ObservedData2012['water_level_C39_offset'][index[0]-1] + ObservedData2012['water_level_C39_offset'][index[-1]+1])

for d1,d2 in zip(ObservedData2012['water_level_C39_offset'], ObservedData2012['water_level_C40_offset']):
    sum = 0
    if d1 < 10:
        sum = 0.5*(d1+d2)
    else:
        sum = 0.5*(d2 + mid_value)
    data.append(sum)
ObservedData2012['water_level_Center_offset'] = np.concatenate(data)

print len(ObservedData2013['water_level_C39_offset']), len(ObservedData2013['water_level_C40'])

data = []
c = 0
for i in range(len(ObservedData2013['water_level_C39_offset'])):
    if i <7:
        data.append(ObservedData2013['water_level_C39_offset'][i])
    elif (i < 113):
        sum = 0.5*(ObservedData2013['water_level_C39_offset'][i] + ObservedData2013['water_level_C40'][i-7])    
        data.append(sum)
    else:
        data.append(ObservedData2013['water_level_C39_offset'][i])


ObservedData2013['water_level_Center'] = np.array(data)

#--- 2014
print len(ObservedData2014['water_level_C39']), len(ObservedData2014['water_level_C40'])
data = []
for i in range(len(ObservedData2014['water_level_C39_offset'])):
        sum = 0.5*(ObservedData2014['water_level_C39_offset'][i] + ObservedData2014['water_level_C40'][i])    
        data.append(sum)

ObservedData2014['water_level_Center'] = np.array(data)

114 106
85 85


In [57]:
def PlotWithCenterAverage():
    outfile = '/Users/ajc/AllPostProcessData/2019/simulations/barrow-iwp/Feb7/'
    fig, axs = plt.subplots(3,3, figsize=(8,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    plt.tight_layout()
    axs = axs.ravel()
    #fig, axs = plt.subplots(3,2,figsize=(10,8))


    ax = plt.subplot(321)
    plt.fill_between(ObservedData2012['time_C37'], ObservedData2012['water_level_C37']-0.03, 
                     ObservedData2012['water_level_C37']+ 0.03, color='c')
    plt.plot(ObservedData2012['time_C37'], ObservedData2012['water_level_C37'],'r')
    plt.plot(DataSim['time_trough'], DataSim['water_level_trough'],'k*',markersize='4')
    plt.plot(DataSim['time'], DataSim['TD_center'],'m',markersize='4')
    plt.axhline(y=Z_t,linestyle='--',color='g')
    plt.text(882, 4.6, 'Trough, 2012')
    plt.ylabel('Water level')

    #ax.set_xticks(np.arange(880, 1001, step=20))
    #ax.set_xticklabels(timeadj2012)
    ax.set_xlim(880, 1000)
    #plt.margins(0.2)
    plt.ylim(4.2, 5.2)


    ax = plt.subplot(322)

    plt.fill_between(ObservedData2012['time_C39'], ObservedData2012['water_level_Center_Elev']-0.03, 
                     ObservedData2012['water_level_Center_Elev']+ 0.03, color='c')
    plt.plot(ObservedData2012['time_C39'], ObservedData2012['water_level_Center_Elev'],'r',label='Center, 2012')
    #plt.plot(ObservedData2012['time_C40'], ObservedData2012['water_level_C40'],'g')#, label='Observed (2014 C37)')
    #plt.plot(ObservedData2012['time_C39_mask'], ObservedData2012['water_level_C39_mask'],'m')#, label='Observed (2014 C37)')
    plt.plot(DataSim['time_center'], DataSim['water_level_center'],'k*',markersize='4',label='Simulated')
    plt.plot(DataSim['time'], DataSim['TD_center'],'m',markersize='4')

    plt.axhline(y=Z_c,linestyle='--',color='g')
    #plt.legend()
    plt.text(882, 4.6, 'Center, 2012')
    plt.xlim(880, 1000)
    plt.ylim(4.2, 5.2)

    ax1 = ax.twinx()
    #ax1.plot(DataSim['time'],DataSim['Rain_precip'],'y')
    ax1.set_ylabel('Precipitation Rain [mm]',color='y')
    ax1.tick_params('y', colors='y')
    ax1.set_ylim([-17.0, 10])
    ax1.set_yticks([0, 5, 10])
    #---------------------------------------------------------
    # ------------ 2013 --------------------------------------
    ax = plt.subplot(323)
    plt.fill_between(ObservedData2013['time_C37'], ObservedData2013['water_level_C37']-0.03, 
                     ObservedData2013['water_level_C37']+ 0.03, color='c')
    plt.plot(ObservedData2013['time_C37'], ObservedData2013['water_level_C37'],'r')#, label='Trough, 2012')
    plt.plot(DataSim['time_trough'], DataSim['water_level_trough'],'k*',markersize='4',label='Simulated')
    plt.plot(DataSim['time'], DataSim['TD_trough'],'m',markersize='4')

    plt.axhline(y=Z_t,linestyle='--',color='g')
    plt.text(1238, 4.6, 'Trough, 2013')
    #plt.xlabel('Day')
    plt.ylabel('Water level')
    plt.xlim(1236, 1362)
    plt.ylim(4.3, 5.1)


    ax = plt.subplot(324)

    plt.fill_between(ObservedData2013['time_C39'], ObservedData2013['water_level_Center']-0.03, 
                     ObservedData2013['water_level_Center']+ 0.03, color='c')
    plt.plot(ObservedData2013['time_C39'], ObservedData2013['water_level_Center'],'r')
    plt.plot(DataSim['time_center'], DataSim['water_level_center'],'k*',markersize='4',label='Simulated')
    plt.plot(DataSim['time'], DataSim['TD_center'],'m',markersize='4')


    plt.text(1238, 4.6, 'Center, 2013')
    plt.axhline(y=Z_c,linestyle='--',color='g')
    #plt.legend()
    plt.xlim(1236, 1362)
    plt.ylim(4.3, 5.1)
    ax1 = ax.twinx()


    #ax1.plot(DataSim['time'],DataSim['Rain_precip'],'y')
    ax1.set_ylabel('Precipitation Rain [mm]',color='y')
    ax1.tick_params('y', colors='y')
    ax1.set_ylim([-17.0, 10])
    ax1.set_yticks([0, 5, 10])


    #---------------- 2014 -----------------------------
    # --------------------------------------------------
    ax = plt.subplot(325)
    plt.fill_between(ObservedData2014['time_C37'], ObservedData2014['water_level_C37']-0.03, 
                     ObservedData2014['water_level_C37']+ 0.03, color='c')
    plt.plot(ObservedData2014['time_C37'], ObservedData2014['water_level_C37'],'r')#, label='Observed (trough) 2012')
    plt.plot(DataSim['time_trough'], DataSim['water_level_trough'],'k*',markersize='4',label='Simulated')
    plt.plot(DataSim['time'], DataSim['TD_trough'],'m',markersize='4')

    plt.axhline(y=Z_t,linestyle='--',color='g')
    plt.text(1622, 4.6, 'Trough, 2014')
    plt.xlabel('Day')
    plt.ylabel('Water level')
    plt.xlim(1620, 1710)
    plt.ylim(4.3, 5.1)
    #plt.legend()

    ax = plt.subplot(326)

    plt.fill_between(ObservedData2014['time_C39'], ObservedData2014['water_level_Center']-0.03, 
                     ObservedData2014['water_level_Center']+ 0.03, color='c')
    plt.plot(ObservedData2014['time_C39'], ObservedData2014['water_level_Center'],'r',label='Observed')
    plt.plot(DataSim['time_center'], DataSim['water_level_center'],'k*',markersize='4',label='Simulated')
    plt.plot(DataSim['time'], DataSim['TD_center'],'m',markersize='4',label='Thaw depth')
    #plt.plot(ObservedData2014['time_C39'], ObservedData2014['water_level_C39'],'r')#, label='Observed (2014 C37)')

    plt.text(1622, 4.6, 'Center, 2014')
    plt.axhline(y=Z_c,linestyle='--',color='g')
    plt.legend()
    plt.xlim(1620, 1710)
    plt.ylim(4.3, 5.1)

    ax1 = ax.twinx()
    #ax1.plot(DataSim['time'],DataSim['Rain_precip'],'y')
    ax1.set_ylabel('Precipitation Rain [mm]',color='y')
    ax1.tick_params('y', colors='y')
    ax1.set_ylim([-17.0, 10])
    ax1.set_yticks([0, 5, 10])

    ax.legend(loc='upper right', fontsize=10, ncol=3, bbox_to_anchor=(0.7,- 0.32, .1, .1))
    plt.tight_layout()
    plt.savefig(outfile+'comparison-%s.pdf'%simulation, bbox_inches='tight', dpi=100)
    plt.show()




In [60]:
def PlotWithoutCenterAverage():
    outfile = '/Users/ajc/AllPostProcessData/2019/simulations/barrow-iwp/Feb7/'
    fig, axs = plt.subplots(3,3, figsize=(8,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    plt.tight_layout()
    axs = axs.ravel()
    #fig, axs = plt.subplots(3,2,figsize=(10,8))


    ax = plt.subplot(321)
    plt.plot(ObservedData2012['time_C37'], ObservedData2012['water_level_C37'],'r')
    plt.plot(DataSim['time_trough'], DataSim['water_level_trough'],'k*',markersize='4')
    plt.plot(DataSim['time'], DataSim['TD_center'],'m',markersize='4')
    plt.axhline(y=Z_t,linestyle='--',color='g')
    plt.text(882, 4.6, 'Trough, 2012')
    plt.ylabel('Water level')

    ax.set_xlim(880, 1000)
    plt.ylim(4.3, 5.1)

    ax = plt.subplot(322)
    plt.plot(ObservedData2012['time_C40'], ObservedData2012['water_level_C40'],'r')
    plt.plot(ObservedData2012['time_C39_mask'], ObservedData2012['water_level_C39_mask'],'g')
    plt.plot(DataSim['time_center'], DataSim['water_level_center'],'k*',markersize='4',label='Simulated')
    plt.plot(DataSim['time'], DataSim['TD_center'],'m',markersize='4')

    plt.axhline(y=Z_c,linestyle='--',color='g')
    #plt.legend()
    plt.text(882, 4.6, 'Center, 2012')
    plt.xlim(880, 1000)
    plt.ylim(4.3, 5.1)
    
    """
    ax1 = ax.twinx()
    ax1.plot(dataPrecip['time_2012'],dataPrecip['rain_2012'],'c', label='Observed rain')
    ax1.plot(DataSim['Time_forcing'],DataSim['Rain_forcing'],'g.', label='Forcing data (rain)')

    ax1.set_ylabel('Precipitation Rain [mm]',color='y')
    ax1.tick_params('y', colors='y')
    #ax1.set_ylim([-17.0, 10])
    #ax1.set_yticks([0, 5, 10])
    """
    
    #---------------------------------------------------------
    # ------------ 2013 --------------------------------------
    ax = plt.subplot(323)
    plt.plot(ObservedData2013['time_C37'], ObservedData2013['water_level_C37'],'r')
    plt.plot(DataSim['time_trough'], DataSim['water_level_trough'],'k*',markersize='4',label='Simulated')
    plt.plot(DataSim['time'], DataSim['TD_trough'],'m',markersize='4')

    plt.axhline(y=Z_t,linestyle='--',color='g')
    plt.text(1238, 4.6, 'Trough, 2013')
    #plt.xlabel('Day')
    plt.ylabel('Water level')
    plt.xlim(1236, 1362)
    plt.ylim(4.3, 5.1)

    ax = plt.subplot(324)
    
    #plt.plot(ObservedData2013['time_C40'], ObservedData2013['water_level_C40'],'r')
    plt.plot(ObservedData2013['time_C39'], ObservedData2013['water_level_C39_offset'],'g')
    plt.plot(DataSim['time_center'], DataSim['water_level_center'],'k*',markersize='4',label='Simulated')
    plt.plot(DataSim['time'], DataSim['TD_center'],'m',markersize='4')


    plt.text(1238, 4.6, 'Center, 2013')
    plt.axhline(y=Z_c,linestyle='--',color='g')
    #plt.legend()
    plt.xlim(1236, 1362)
    plt.ylim(4.3, 5.1)
    ax1 = ax.twinx()

    """
    ax1.plot(dataPrecip['time_2013'],dataPrecip['rain_2013'],'c', label='Observed rain')
    ax1.plot(DataSim['Time_forcing'],DataSim['Rain_forcing'],'g.', label='Forcing data (rain)')

    ax1.set_ylabel('Precipitation Rain [mm]',color='y')
    ax1.tick_params('y', colors='y')
    #ax1.set_ylim([-17.0, 10])
    #ax1.set_yticks([0, 5, 10])
    """

    #---------------- 2014 -----------------------------
    # --------------------------------------------------
    ax = plt.subplot(325)
    plt.plot(ObservedData2014['time_C37'], ObservedData2014['water_level_C37'],'r')
    plt.plot(DataSim['time_trough'], DataSim['water_level_trough'],'k*',markersize='4',label='Simulated')
    plt.plot(DataSim['time'], DataSim['TD_trough'],'m',markersize='4')

    plt.axhline(y=Z_t,linestyle='--',color='g')
    plt.text(1622, 4.6, 'Trough, 2014')
    plt.xlabel('Day')
    plt.ylabel('Water level')
    plt.xlim(1620, 1710)
    plt.ylim(4.3, 5.1)

    ax = plt.subplot(326)

    #pondedepth40 = [x - 4.91 for x in ObservedData2014['water_level_C40']]
    #plt.plot(ObservedData2014['time_C40'], ObservedData2014['water_level_C40'],'r',label='Observed (C40)')
    plt.plot(ObservedData2014['time_C39'], ObservedData2014['water_level_C39_offset'],'g',label='Observed (C39)')
    plt.plot(DataSim['time_center'], DataSim['water_level_center'],'k*',markersize='4',label='Simulated')
    plt.plot(DataSim['time'], DataSim['TD_center'],'m',markersize='4',label='Thaw depth')

    plt.text(1622, 4.6, 'Center, 2014')
    plt.axhline(y=Z_c,linestyle='--',color='g')
    plt.legend()
    plt.xlim(1620, 1710)
    plt.ylim(4.3, 5.1)
    
    """
    ax1 = ax.twinx()
    ax1.plot(dataPrecip['time_2014'],dataPrecip['rain_2014'],'c', label='Observed rain')
    ax1.plot(DataSim['Time_forcing'],DataSim['Rain_forcing'],'g.', label='Forcing data (rain)')
    
    ax1.set_ylabel('Precipitation Rain [mm]',color='y')
    ax1.tick_params('y', colors='y')
    #ax1.set_ylim([-17.0, 10])
    #ax1.set_yticks([0, 5, 10])
    """
    ax.legend(loc='upper right', fontsize=7, ncol=4, bbox_to_anchor=(0.0,-0.32, .1, .1))
    #ax1.legend(loc='upper right', fontsize=7, ncol=2, bbox_to_anchor=(0.8,- 0.32, .1, .1))
    plt.tight_layout()
    plt.savefig(outfile+'%s-2pw.pdf'%simulation, bbox_inches='tight', dpi=100)
    plt.show()

In [61]:
#PlotWithCenterAverage()
PlotWithoutCenterAverage()

In [51]:
#print np.sum(DataSim['Rain_precip'])
print np.sum(DataSim['Rain_forcing'])
print np.sum(dataPrecip['rain_2012'])
print dataPrecip['time_2012'][:2]

KeyError: 'Rain_forcing'

In [ ]:
"""
pondedepth40 = [(x - 4.91)*100 for x in ObservedData2014['water_level_C40']]
pondedepth39 = [(x - 4.81)*100 for x in ObservedData2014['water_level_C39']]
pondedepth37 = [(x - 4.72)*100 for x in ObservedData2014['water_level_C37']]
plt.plot(ObservedData2014['time_C37'], pondedepth37,'k',label='Observed (C37)')
plt.plot(ObservedData2014['time_C39'], pondedepth39,'g',label='Observed (C39)')
plt.plot(ObservedData2014['time_C40'], pondedepth40,'r',label='Observed (C40)')
plt.axhline(y=0,linestyle='--',color='b')
"""

pondedepth40 = [(x - 4.91*0)*1 for x in ObservedData2012['water_level_C40']]
pondedepth39 = [(x - 4.81*0)*1 for x in ObservedData2012['water_level_C39_mask']]
pondedepth37 = [(x - 4.72*0)*1 for x in ObservedData2012['water_level_C37']]
#pondedepth40sim = [(x - 4.81)*100 for x in DataSim['water_level_center'] ]
plt.plot(ObservedData2012['time_C37'], pondedepth37,'k',label='Observed (C37)')
plt.plot(ObservedData2012['time_C39_mask'], pondedepth39,'g',label='Observed (C39)')
plt.plot(ObservedData2012['time_C40'], pondedepth40,'r',label='Observed (C40)')
#plt.plot(DataSim['time_center'], pondedepth40sim,'m',label='Simulated center')
#plt.axhline(y=0,linestyle='--',color='b')
plt.ylabel('water table')
plt.xlabel('Time (2012)')
plt.legend()
plt.show()

In [ ]:
reload(SiteC_2012_2014WL)

WellsData2012 = SiteC_2012_2014WL.getDataAllWells(2012)
WellsData2013 = SiteC_2012_2014WL.getDataAllWells(2013)
WellsData2014 = SiteC_2012_2014WL.getDataAllWells(2014)

In [ ]:
#outfile = '/Users/ajc/AllPostProcessData/2019/simulations/barrow-iwp/Feb4/'
print WellsData2012.keys()
fig, axs = plt.subplots(3,1, figsize=(8,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
plt.tight_layout()
axs = axs.ravel()

ax = plt.subplot(311)

plt.plot(WellsData2012['time_C37'], WellsData2012['water_level_C37'],'r')
plt.plot(WellsData2012['time_C39'], WellsData2012['water_level_C39'],'k.')
plt.plot(WellsData2012['time_C41'], WellsData2012['water_level_C41'],color='b')
plt.plot(WellsData2012['time_C43'], WellsData2012['water_level_C43'],color='c')
plt.axhline(y=4.81,linestyle='--',color='k')
plt.axhline(y=4.72,linestyle='--',color='c')
plt.ylabel('water level [m]')
plt.xlabel('Time')

ax = plt.subplot(312)

plt.plot(WellsData2013['time_C37'], WellsData2013['water_level_C37'],'r')
plt.plot(WellsData2013['time_C39'], WellsData2013['water_level_C39'],'k.')
plt.plot(WellsData2013['time_C41'], WellsData2013['water_level_C41'],color='b')
plt.plot(WellsData2013['time_C43'], WellsData2013['water_level_C43'],color='c')
plt.axhline(y=4.81,linestyle='--',color='k')
plt.axhline(y=4.72,linestyle='--',color='c')
plt.ylabel('water level [m]')
plt.xlabel('Time')

ax = plt.subplot(313)

plt.plot(WellsData2014['time_C37'], WellsData2014['water_level_C37'],'r',label='C37')
plt.plot(WellsData2014['time_C39'], WellsData2014['water_level_C39'],'k.',label='C39')
plt.plot(WellsData2014['time_C41'], WellsData2014['water_level_C41'],color='b',label='C41')
plt.plot(WellsData2014['time_C43'], WellsData2014['water_level_C43'],color='c',label='C43')

plt.axhline(y=4.81,linestyle='--',color='k')
plt.axhline(y=4.72,linestyle='--',color='c')

plt.ylabel('water level [m]')
plt.xlabel('Time')
plt.legend()
plt.show()